In [164]:
import yaml
from run_experiment import choose_dataset_class,choose_task_classes

import torch
import numpy as np
from collections import Counter

In [10]:
# 0. read the embeddings and labesl
yaml_args= yaml.load(open('../SA-config/en_ewt-pos-get_cor.yaml'))

dataset_class = choose_dataset_class(yaml_args)

task_class, reporter_class, loss_class = choose_task_classes(yaml_args)

task = task_class(yaml_args)

expt_dataset = dataset_class(yaml_args, task)

In [64]:
# 1. prepare the label matrix.
labels=[]
for obser in train_data.observations:
    labels.append(task.labels(obser))

all_labels = torch.cat(labels, 0).numpy().astype(int)

all_labels.shape
all_labels_mat= np.zeros((all_labels.size, all_labels.max()+1))
all_labels_mat[np.arange(all_labels.size),all_labels] = 1
all_labels_mat.shape

(204609, 50)

In [53]:
# 2. prepare the embedding matrix.

embeddings=[]
for obser in train_data.observations:
    embeddings.append(obser.embeddings)

all_embeddings = torch.cat(embeddings, 0).numpy()

all_embeddings.shape

(204609, 768)

In [137]:
# 3. compute the corrcoef matrix. 
# num=204609
# corr_mat_whole = np.corrcoef(all_labels_mat[0:num].T, all_embeddings[0:num].T)
corr_mat_whole = np.corrcoef(all_labels_mat.T, all_embeddings.T)
corr_mat=np.absolute(corr_mat_whole[0:50,50:])
corr_mat.shape

(50, 768)

In [ ]:
# 4. output the corr_mat data. dump as a pkl.


In [195]:
# 4. output the corr_sum_dim
# we compute the corr score of each dim for all the 50 pos labels.
dim_sum=[]
for i in range(768):
    dim_sum.append(np.sum(corr_mat[:,i]))

corr_sum_dim=np.argsort(dim_sum)[::-1]
with open('../SA-dim-files/average_corr_dim.tsv','w') as fout:
    for dim in corr_sum_dim:
        fout.write('{} '.format(dim))

In [193]:
# 5. output the weighted_average_corr_dim
# we compute the weighted average corr socre of each dim for all the pos labels

# 5.1 get the distribution/weights 
all_labels
result = Counter(all_labels)
weights = [result[i]/len(all_labels) for i in range(50)]



# 5.2 get the weighted score
dim_weighted_average=[]
for i in range(768):
    dim_weighted_average.append(np.average(corr_mat[:,i],weights=weights))
    
# 5.3 output the dims
dim_weighted_average_dim=np.argsort(dim_weighted_average)[::-1]
with open('../SA-dim-files/weighted_average_corr_dim.tsv','w') as fout:
    for dim in dim_weighted_average_dim:
        fout.write('{} '.format(dim))

In [194]:
# 6. output the max_corr
max_val=[]
for i in range(768):
    max_val.append(np.max(corr_mat[:,i]))

dim_max_val=np.argsort(max_val)[::-1]
with open('../SA-dim-files/max_corr_dim.tsv','w') as fout:
    for dim in dim_weighted_average_dim:
        fout.write('{} '.format(dim))

In [191]:
np.max(dim_sum),np.max(dim_weighted_average),np.max(corr_mat),np.max(max_val)

(0.08341785709925074,
 0.19993299987706492,
 0.4437054850588246,
 0.4437054850588246)